In [23]:
# Import all the necessary libraries

import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline

In [24]:
# Read data from the CSV into a dataframe

data_season_1 = pd.read_csv('1011.csv')
data_season_2 = pd.read_csv('1112.csv')
data_season_3 = pd.read_csv('1213.csv')
data_season_4 = pd.read_csv('1314.csv')
data_season_5 = pd.read_csv('1415.csv')
data_season_6 = pd.read_csv('1516.csv')
data_season_7 = pd.read_csv('1617.csv')
data_season_8 = pd.read_csv('1718.csv')
data_season_9 = pd.read_csv('1819.csv')
data_season_current = pd.read_csv('1920.csv')


In [25]:
#Gets all the statistics related to gameplay
                      
needed_cols = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']

stats_1 = data_season_1[needed_cols]                      
stats_2 = data_season_2[needed_cols]
stats_3 = data_season_3[needed_cols]
stats_4 = data_season_4[needed_cols]
stats_5 = data_season_5[needed_cols]
stats_6 = data_season_6[needed_cols]
stats_7 = data_season_7[needed_cols]
stats_8 = data_season_8[needed_cols]
stats_9 = data_season_9[needed_cols]
stats_current = data_season_current[needed_cols]


In [26]:
# Gets the goals scored agg arranged by teams and matchweek
def goals_scored_total(additional_stats):
    # Create a dictionary with team names as keys
    teams = {}
    for i in additional_stats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(additional_stats)):
        HTGS = additional_stats.iloc[i]['FTHG']
        ATGS = additional_stats.iloc[i]['FTAG']
        teams[additional_stats.iloc[i].HomeTeam].append(HTGS)
        teams[additional_stats.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Gets the goals conceded agg arranged by teams and matchweek
def goals_conceded_total(additional_stats):
    # Create a dictionary with team names as keys
    teams = {}
    for i in additional_stats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(additional_stats)):
        ATGC = additional_stats.iloc[i]['FTHG']
        HTGC = additional_stats.iloc[i]['FTAG']
        teams[additional_stats.iloc[i].HomeTeam].append(HTGC)
        teams[additional_stats.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def goal_scoring_stats(additional_stats):
    GC = goals_conceded_total(additional_stats)
    GS = goals_scored_total(additional_stats)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = additional_stats.iloc[i].HomeTeam
        at = additional_stats.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    additional_stats['HTGS'] = HTGS
    additional_stats['ATGS'] = ATGS
    additional_stats['HTGC'] = HTGC
    additional_stats['ATGC'] = ATGC
    
    return additional_stats


# Apply to each dataset
stats_1 = goal_scoring_stats(stats_1)
stats_2 = goal_scoring_stats(stats_2)
stats_3 = goal_scoring_stats(stats_3)
stats_4 = goal_scoring_stats(stats_4)
stats_5 = goal_scoring_stats(stats_5)
stats_6 = goal_scoring_stats(stats_6)
stats_7 = goal_scoring_stats(stats_7)
stats_8 = goal_scoring_stats(stats_8)
stats_9 = goal_scoring_stats(stats_9)
stats_current = goal_scoring_stats(stats_current)



C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [27]:
def team_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def cuml_team_points(match_result):
    match_result_points = match_result.applymap(team_points)
    for i in range(2,39):
        match_result_points[i] = match_result_points[i] + match_result_points[i-1]
        
    match_result_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return match_result_points


def team_results(additional_stats):
    # Create a dictionary with team names as keys
    teams = {}
    for i in additional_stats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(additional_stats)):
        if additional_stats.iloc[i].FTR == 'H':
            teams[additional_stats.iloc[i].HomeTeam].append('W')
            teams[additional_stats.iloc[i].AwayTeam].append('L')
        elif additional_stats.iloc[i].FTR == 'A':
            teams[additional_stats.iloc[i].AwayTeam].append('W')
            teams[additional_stats.iloc[i].HomeTeam].append('L')
        else:
            teams[additional_stats.iloc[i].AwayTeam].append('D')
            teams[additional_stats.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def total_team_points(additional_stats):
    match_result = team_results(additional_stats)
    cuml_points = cuml_team_points(match_result)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = additional_stats.iloc[i].HomeTeam
        at = additional_stats.iloc[i].AwayTeam
        HTP.append(cuml_points.loc[ht][j])
        ATP.append(cuml_points.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    additional_stats['HTP'] = HTP
    additional_stats['ATP'] = ATP
    return additional_stats
    
# Apply to each dataset
stats_1 = total_team_points(stats_1)
stats_2 = total_team_points(stats_2)
stats_3 = total_team_points(stats_3)
stats_4 = total_team_points(stats_4)
stats_5 = total_team_points(stats_5)
stats_6 = total_team_points(stats_6)
stats_7 = total_team_points(stats_7)
stats_8 = total_team_points(stats_8)
stats_9 = total_team_points(stats_9)
stats_current = total_team_points(stats_current)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
def current_form(additional_stats,num):
    form = team_results(additional_stats)
    form_def = form.copy()
    for i in range(num,39):
        form_def[i] = ''
        j = 0
        while j < num:
            form_def[i] += form[i-j]
            j += 1           
    return form_def

def count_form(additional_stats,num):
    form = current_form(additional_stats,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = additional_stats.iloc[i].HomeTeam
        at = additional_stats.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    additional_stats['HM' + str(num)] = h                 
    additional_stats['AM' + str(num)] = a

    
    return additional_stats


def count_form_df(stats):
    stats = count_form(stats,1)
    stats = count_form(stats,2)
    stats = count_form(stats,3)
    stats = count_form(stats,4)
    stats = count_form(stats,5)
    return stats    
    
# Make changes to df
stats_1 = count_form_df(stats_1)
stats_2 = count_form_df(stats_2)
stats_3 = count_form_df(stats_3)
stats_4 = count_form_df(stats_4)
stats_5 = count_form_df(stats_5)
stats_6 = count_form_df(stats_6)
stats_7 = count_form_df(stats_7)
stats_8 = count_form_df(stats_8)
stats_9 = count_form_df(stats_9)
stats_current = count_form_df(stats_current)


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

stats_1 = stats_1[cols]
stats_2 = stats_2[cols]
stats_3 = stats_3[cols]
stats_4 = stats_4[cols]
stats_5 = stats_5[cols]
stats_6 = stats_6[cols]
stats_7 = stats_7[cols]
stats_8 = stats_8[cols]
stats_9 = stats_9[cols]
stats_current = stats_current[cols]

In [43]:
prev_positions = pd.read_csv("leaguepositions.csv")
prev_positions.set_index(['Team'], inplace=True)
prev_positions = prev_positions.fillna(18)

def last_season_pos(additional_stats, prev_positions, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = additional_stats.iloc[i].HomeTeam
        at = additional_stats.iloc[i].AwayTeam
        HomeTeamLP.append(prev_positions.loc[ht][year])
        AwayTeamLP.append(prev_positions.loc[at][year])
    additional_stats['HomeTeamLP'] = HomeTeamLP
    additional_stats['AwayTeamLP'] = AwayTeamLP
    return additional_stats

stats_1 = last_season_pos(stats_1, prev_positions, 0)
stats_2 = last_season_pos(stats_2, prev_positions, 1)
stats_3 = last_season_pos(stats_3, prev_positions, 2)
stats_4 = last_season_pos(stats_4, prev_positions, 3)
stats_5 = last_season_pos(stats_5, prev_positions, 4)
stats_6 = last_season_pos(stats_6, prev_positions, 5)
stats_7 = last_season_pos(stats_7, prev_positions, 6)
stats_8 = last_season_pos(stats_8, prev_positions, 7)
stats_9 = last_season_pos(stats_9, prev_positions, 8)

In [44]:
def mw(additional_stats):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    additional_stats['MW'] = MatchWeek
    return additional_stats

stats_1 = mw(stats_1)
stats_2 = mw(stats_2)
stats_3 = mw(stats_3)
stats_4 = mw(stats_4)
stats_5 = mw(stats_5)
stats_6 = mw(stats_6)
stats_7 = mw(stats_7)
stats_8 = mw(stats_8)
stats_9 = mw(stats_9)
stats_current = mw(stats_current)

In [47]:
additional_stats = pd.concat([stats_1,
                              stats_2,
                              stats_3,
                              stats_4,
                              stats_5,
                              stats_6,
                              stats_7,
                              stats_8,
                              stats_9,
                              stats_current], ignore_index=True, sort=False)


# Gets the form points.
def points_form(string):
    sum = 0
    for letter in string:
        sum += team_points(letter)
    return sum

additional_stats['HTFormPtsStr'] = additional_stats['HM1'] + additional_stats['HM2'] + additional_stats['HM3'] + additional_stats['HM4'] + additional_stats['HM5']
additional_stats['ATFormPtsStr'] = additional_stats['AM1'] + additional_stats['AM2'] + additional_stats['AM3'] + additional_stats['AM4'] + additional_stats['AM5']

additional_stats['HTFormPts'] = additional_stats['HTFormPtsStr'].apply(points_form)
additional_stats['ATFormPts'] = additional_stats['ATFormPtsStr'].apply(points_form)

# Identify Win/Loss Streaks if any.
def winstreak_3(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def winstreak_5(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def losestreak_3(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def losestreak_5(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
additional_stats['HTWinStreak3'] = additional_stats['HTFormPtsStr'].apply(winstreak_3)
additional_stats['HTWinStreak5'] = additional_stats['HTFormPtsStr'].apply(winstreak_5)
additional_stats['HTLossStreak3'] = additional_stats['HTFormPtsStr'].apply(losestreak_3)
additional_stats['HTLossStreak5'] = additional_stats['HTFormPtsStr'].apply(losestreak_5)

additional_stats['ATWinStreak3'] = additional_stats['ATFormPtsStr'].apply(winstreak_3)
additional_stats['ATWinStreak5'] = additional_stats['ATFormPtsStr'].apply(winstreak_5)
additional_stats['ATLossStreak3'] = additional_stats['ATFormPtsStr'].apply(losestreak_3)
additional_stats['ATLossStreak5'] = additional_stats['ATFormPtsStr'].apply(losestreak_5)

additional_stats.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [48]:
# Get Goal Difference
additional_stats['HTGD'] = additional_stats['HTGS'] - additional_stats['HTGC']
additional_stats['ATGD'] = additional_stats['ATGS'] - additional_stats['ATGC']

# Diff in points
additional_stats['DiffPts'] = additional_stats['HTP'] - additional_stats['ATP']
additional_stats['DiffFormPts'] = additional_stats['HTFormPts'] - additional_stats['ATFormPts']

additional_stats['DiffLP'] = additional_stats['HomeTeamLP'] - additional_stats['AwayTeamLP']


In [49]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
additional_stats.MW = additional_stats.MW.astype(float)

for col in cols:
    additional_stats[col] = additional_stats[col] / additional_stats.MW

In [50]:
#def hw(string):
#    if string == 'H':
#        return 'H'
#    else:
#        return 'NH'
    
#additional_stats['FTR'] = additional_stats.FTR.apply(only_hw)

# Testing set (2019-20 season)
additional_stats_test = additional_stats[3420:]

In [51]:
additional_stats.to_csv("TESTfinal_dataset.csv")
additional_stats_test.to_csv("test1.csv")